In [1]:
# Imported Libraries
import pandas as pd
import pymysql

# Data visualisation tools for later.
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from apyori import apriori

# Variables
dumpDirectory = "./TableDump/"

#######Connection to Client Database
host="qut-ds.c2m09j1oykve.ap-southeast-2.rds.amazonaws.com"
port=4005
dbname="qut_ds1"
user="qut_ds1"
password="GaAVSqC#9JR8"
conn = pymysql.connect(host=host, user=user,port=port,password=password, db=dbname)


print(conn)

In [3]:
####Checking queries

###Customer Personal details
##  add the desired rows in the query from the three tables
df_customer_details = pd.read_sql("select s.u_disabilities, s.u_gender, s.u_date_of_birth, r.SA1, r.SA2, r.SA3,r.SA4  from SNOW_csm_consumer_user s left join  HH_member h  on s.u_ndis_number = h.membership_number left join libe_leapinprod_memberregion r on r.MemberId =  s.u_leapin_id where s.u_stage = 'li_managed' and s.u_ndis_number is not null;", con=conn) 
print(df_customer_details.info())
print(df_customer_details)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6241 entries, 0 to 6240
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   u_disabilities   6241 non-null   object 
 1   u_gender         6241 non-null   object 
 2   u_date_of_birth  6239 non-null   object 
 3   SA1              6021 non-null   float64
 4   SA2              6021 non-null   float64
 5   SA3              6021 non-null   float64
 6   SA4              6021 non-null   float64
dtypes: float64(4), object(3)
memory usage: 341.4+ KB
None
            u_disabilities u_gender u_date_of_birth        SA1      SA2  \
0                                        1922-11-30  3100614.0  31006.0   
1                                        2007-04-06  1144614.0  11446.0   
2                            Female      1954-03-26        0.0      0.0   
3                            Female      1978-12-24  3138504.0  31385.0   
4                                        1995-

In [7]:
###Invoice claim details
##  add the desired rows in the query from the three tables
df_invoice_claim_details = pd.read_sql("SELECT c.invoice_id, c.state, c.risk_score, c.risk_level, c.risk_warnings, c.unit_price, c.start_date, c.end_date, c.invoiced_units, c.invoiced_unit_price, c.claimed_units, c.claimed_unit_price, c.claimed_date, c.funded_amount, c.funded_date, c.submitted_date, c.approved_date, c.received_date, c.processed_status, c.reconcile_status, c.ndis_cancelled_date, c.cancel_offset_z4_no, i.state, i.is_pre_approved, i.is_resolved, i.is_risk_identified, i.risk_score, i.invoice_total, i.funded_total, i.funded_date, i.claimed_date, i.reviewed_date, i.invoice_date, i.approved_date, i.submitted_date, i.received_date, m.membership_number, m.status, m.risk_factor, m.price_zone_code FROM HH_claim c left join HH_invoice i on c.invoice_id = i.id left join HH_member m on i.member_id = m.id ;", con=conn)
print(df_invoice_claim_details.info())
print(df_invoice_claim_details)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968874 entries, 0 to 968873
Data columns (total 40 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   invoice_id           968874 non-null  int64         
 1   state                968874 non-null  object        
 2   risk_score           968874 non-null  float64       
 3   risk_level           968874 non-null  object        
 4   risk_warnings        41285 non-null   object        
 5   unit_price           968874 non-null  float64       
 6   start_date           968874 non-null  object        
 7   end_date             968874 non-null  object        
 8   invoiced_units       966054 non-null  float64       
 9   invoiced_unit_price  968874 non-null  float64       
 10  claimed_units        965796 non-null  float64       
 11  claimed_unit_price   968442 non-null  float64       
 12  claimed_date         968328 non-null  object        
 13  funded_amount 

In [2]:


# 1/ a table where all information of service providers, their services

# Method - SNOW_customer_account merged with u_service_type.csv

# Service Types - Multi-valued field 
df_dump = pd.read_csv(dumpDirectory + "u_service_type.csv", keep_default_na = False, na_values = ["", "nan"])
# print(df_dump.info())

directoryQuery = "select sys_id, u_can_provide_the_following_service_types from SNOW_customer_account where u_can_provide_the_following_service_types != ''"

# Directory Provider
df_directory_providers = pd.read_sql(directoryQuery, con=conn)
# print(df_directory_providers.info())

# Split multi-value column into single-valued rows
df_temp = pd.DataFrame(df_directory_providers["u_can_provide_the_following_service_types"].str.split(",").tolist(), index=df_directory_providers.sys_id).stack()
df_temp = df_temp.reset_index([0, "sys_id"])
df_temp.columns = ["sys_id", "u_can_provide_the_following_service_types"]
# df_temp.head(10)

# Join with dump table. CHECK IF CORRECT
df_final = df_temp.set_index("u_can_provide_the_following_service_types").join(df_dump.set_index("sys_id"), how="outer")

print(df_final.info())

# Problem: u_categories_list returns nan upon detecting hex values
# df_final.head(50)

df_final.to_csv("service_providers_info.csv", index=False)







<class 'pandas.core.frame.DataFrame'>
Index: 6472 entries, 00a989b9dba754105229e126 to fdc2213d1bea5c50908033fccc4bcb47
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sys_id             6469 non-null   object
 1   u_categories_list  4164 non-null   object
 2   u_service_types    6040 non-null   object
dtypes: object(3)
memory usage: 202.2+ KB
None


8c7f49619ad,,Early Childhood Early Intervention\r\n35c9c0bbdb0b18508df5d8c7f496196c,,Early Childhood Early Intervention\r\nc0eb85c5db1f5c508df5d8c7f49619e2,,Early Childhood Early Intervention\r\n2406dc13db9fd01067de48370596191c,,Early Childhood Early Intervention\r\n1af0515fdb93141067de483705961909,,Early Childhood Early Intervention\r\n980f2d79db6b54105229e12605961979,,Early Childhood Early Intervention\r\n8e232306db63d4105229e1260596193a,,Early Childhood Early Intervention\r\n474bc0b3dba75c10d4d3a58405961993,,Early Childhood Early Intervention\r\n6fbd52d4dbf7dc105229e12605961912,,Early Childhood Early Intervention\r\na49a2358db7fdc105229e126059619da,,Early Childhood Early Intervention\r\nd04cebb0db0ca410d4d3a584059619d6,,Early Childhood Early Intervention\r\n5fd04468db582450d4d3a58405961956,,Early Childhood Early Intervention\r\n291daacadb74e4105229e12605961999,,Early Childhood Early Intervention\r\ne028e00cdb45ec10d4d3a584059619f8,,Early Childhood Early Intervention\r\n801d3228db4d6